### Merge ethograms between Jay's handlabels and jaaba preds

In [13]:
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
import numpy as np
from tqdm import tqdm
from typing import *
import shutil

In [2]:
df = pd.read_hdf('/data/caitlin/exactly1_hand_labels.hdf')

In [3]:
df

,mat_path,hand_labels,jaaba_labels,merged_ethogram
0,M232_20170306,"{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170314,"{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170328,"{'M234_20170328_v002': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170328_v002': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170328_v015': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
445,M240_20170724FinalChecked,"{'M240_20170724_v014': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170724_v014': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170724_v014': [[0, 0, 0, 0, 0, 0, 0, ..."
446,M240_20170725FinalChecked,"{'M240_20170725_v031': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170725_v031': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170725_v031': [[0, 0, 0, 0, 0, 0, 0, ..."
447,M240_20170727FinalChecked,"{'M240_20170727_v007': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170727_v007': [[0, 0, 0, 0, 0, 0, 0, ...",{}
448,M240_20170802FinalChecked,"{'M240_20170802_v017': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170802_v017': [[0, 0, 0, 0, 0, 0, 0, ...",{}


### Get jaaba ethogram from mat file

In [6]:
#mat_loc = Path('/data/caitlin/mat_files/')

In [7]:
# errors2 = list()
# def get_ethogram2(trial_index: int, mat_path):
#         """
#         Returns the ethogram for a given trial in a session.
#         """
#         m = loadmat(mat_path)
#         try:
#             behaviors = sorted([b.split('_')[0] for b in m['data1'].dtype.names if 'scores' in b])
#         except KeyError:
#             errors2.append(mat_path)
#             return

#         all_behaviors = [
#             "Lift",
#             "Handopen",
#             "Grab",
#             "Sup",
#             "Atmouth",
#             "Chew"
#         ]

#         sorted_behaviors = [b for b in all_behaviors if b in behaviors]

#         ethograms = []

#         mat_trial_index = np.argwhere(m["data1"]["trial"].ravel() == (trial_index + 1))
#         # Trial not found in JAABA data
#         if mat_trial_index.size == 0:
#             return False

#         try:
#             mat_trial_index = mat_trial_index.item()
#         except ValueError:
#             return

      
#         for b in sorted_behaviors:
#             behavior_index = m['data1'].dtype.names.index(f'{b}_postprocessed')
#             row = m['data1'][mat_trial_index][0][behavior_index]
#             row[row == -1] = 0
#             ethograms.append(row)

#         sorted_behaviors = [b.lower() for b in sorted_behaviors]

#         return np.hstack(ethograms).T

In [8]:
# for row in tqdm(df.iterrows()):
#     for key in tqdm(row[1]["hand_labels"].keys()):
#         row[1]["jaaba_labels"][key] = get_ethogram(trial_index=int(key.split('_v')[-1]), 
#                                                    mat_path=mat_loc.joinpath(row[1]["mat_path"]).with_suffix('.mat'))

In [9]:
# errors

In [10]:
# for mat in tqdm(errors):
#     for key in tqdm(df[df['mat_path'] == mat.stem]['jaaba_labels'].item().keys()):
#         df[df['mat_path'] == mat.stem]['jaaba_labels'].item()[key] = get_ethogram2(trial_index=int(key.split('_v')[-1]),
#                                                                                    mat_path=mat)

### want to merge ethograms so that single hand-labeled behavior replaces jabba pred for that behavior

In [11]:
# df["merged_ethogram"] = [dict() for i in range(len(df.index))]

In [30]:
# errors = list()
# for row in tqdm(df.iterrows()):
#     for key in tqdm(row[1]["hand_labels"].keys()):
#         if isinstance(row[1]["jaaba_labels"][key], Union[bool, None]):
#             errors.append((row[1]["mat_path"], key))
#             continue
#         if row[1]["hand_labels"][key].shape != row[1]["jaaba_labels"][key].shape:
#             errors.append((row[1]["mat_path"], key))
#             continue
#         merged_ethogram = row[1]["jaaba_labels"][key]
#         for i, h_row in enumerate(row[1]["hand_labels"][key]):
#             if h_row.any():
#                 merged_ethogram[i] = h_row
#             else:
#                 continue
#         row[1]["merged_ethogram"][key] = merged_ethogram

In [4]:
corrected_trials = list()
count = 0
for row in df.iterrows():
    for key in row[1]["merged_ethogram"].keys():
        if row[1]["merged_ethogram"][key].any():
            count += 1
            corrected_trials.append(key)
count

2208

In [5]:
corrected_trials[0:5]

['M232_20170306_v024',
 'M232_20170306_v059',
 'M232_20170306_v074',
 'M232_20170306_v075',
 'M232_20170306_v076']

### want to download videos for trials that have been correct to `/data/caitlin/potential_ground_truth/` for review

In [6]:
mat_files = Path('/data/caitlin/mat_files/')

In [7]:
vid_paths = list()
for row in tqdm(df.iterrows()):
    m = loadmat(mat_files.joinpath(row[1]['mat_path']).with_suffix('.mat'))
    keys = row[1]["merged_ethogram"].keys()
    for k in keys:
        index = int(k.split('_v')[-1]) - 1
        try:
            vid_paths.append((Path(*Path(m['data']['id'][index][0][0]).parts[:-1])))
        except (KeyError, IndexError):
            try: 
                vid_paths.append((Path(*Path(m['data1']['id'][index][0][0]).parts[:-1])))
            except (KeyError, IndexError):
                vid_paths.append(None)
                continue

450it [00:58,  7.67it/s]


In [8]:
len(corrected_trials)

2208

In [9]:
len(vid_paths)

2208

In [20]:
final_paths = list()
home_path = '/home/clewis7/wasabi/hantmanlab/from_tier2'
for (vp, trial) in zip(vid_paths, corrected_trials):
    if vp == None:
        final_paths.append(None)
        continue
    full_path = home_path + str(vp)
    final_paths.append(Path(full_path).joinpath(trial, trial).with_name('movie_comb').with_suffix('.avi'))

In [21]:
len(final_paths)

2208

In [28]:
ground = Path('/data/caitlin/potential_ground_truth/')

In [ ]:
for vp, trial in tqdm(zip(final_paths, corrected_trials)):
    try:
        shutil.copy(src=vp, dst=ground.joinpath(f"{trial}.avi"))
    except FileNotFoundError:
        print(vp) 

705it [11:48,  1.17it/s]